In [43]:
from datetime import datetime, timedelta
import pandas as pd

from initialize import initialize_client
import regex as re

func_suffix = "OptimizedNew"

In [2]:
ts_input_name = "VAL_11-LT-95034A:X.Value"
ts_output_name = f"VAL_11-LT-95034A:X.CDF.D.AVG.LeakValue{func_suffix}"
tank_volume = 1400
derivative_value_excl = 0.002
# start_date = datetime(2023, 3, 21, 1, 0, 0)

data_dict = {'tot_days':0, 'tot_minutes':15, # convert date to str to make it JSON serializable
            'ts_input_name':ts_input_name, 'ts_output_name':ts_output_name,
            'derivative_value_excl':derivative_value_excl, 'tank_volume':tank_volume,
            'run_sandbox':True, 'dataset_id': 3197476513083188} # NB: change dataset id when going to dev/test/prod!

In [3]:
client = initialize_client(data_dict["run_sandbox"], cache_token=True)

In [4]:
if data_dict["run_sandbox"]:
    cron_interval = "15" # 15
else:
    cron_interval = "15"
    func_suffix = "Deploy"

In [5]:
# Create function
func_drainage = client.functions.create(
    name=f"avg-drainage-rate-{func_suffix}",
    external_id=f"avg-drainage-rate-{func_suffix}",
    # function_handle=handle,
    folder="."
)

In [6]:
func_drainage = client.functions.retrieve(external_id=f"avg-drainage-rate-{func_suffix}")
func_drainage.status

'Deploying'

In [7]:
# Call function - IF NOT RUNNING SCHEDULE!
call_func_drainage = func_drainage.call(data=data_dict)

In [8]:
# Schedule
func_drainage_schedule = client.functions.schedules.create(
    name=f"avg-drainage-rate-schedule-{func_suffix}",
    cron_expression=f"*/{cron_interval} * * * *", # every 15 min
    function_id=func_drainage.id,
    #client_credentials=client,
    description="Calculation scheduled every hour",
    data=data_dict
)

In [ ]:
client.time_series.delete(external_id=f"VAL_11-LT-95034A:X.CDF.D.AVG.LeakValue{func_suffix}")

## Experimental

In [11]:
cc = client.time_series.list(name="VAL_11-LT-95034A:X.CDF.D.AVG.LeakValueOptimizedNew").to_pandas()
not cc.empty

True

In [ ]:
c = initialize_client(False, cache_token=True)

In [ ]:
pd.to_datetime(pd.Timestamp.now().date())

In [ ]:
# c.time_series.data.retrieve(id=215671431499, start="70w-ago", end="60w-ago")
from cognite.client.data_classes import TimeSeries
# c.time_series.create(TimeSeries(name="write_test", external_id="write_test", data_set_id=2391673923486590))
df = pd.DataFrame({"pi:37670": [1,2,3,4]})
df.index = [datetime(2023,7,1,1,0,0), datetime(2023,7,2,1,0,0), datetime(2023,7,3,1,0,0), datetime(2023,7,4,1,0,0)]
# c.time_series.data.insert_dataframe(df) # NOT ALLOWED
c.events.retrieve(id=9007199230377491)

In [ ]:
from cognite.client.data_classes import EventFilter
# All events linked to either primary asset (7681193400980395) or any of its children assets
evtest = c.events.list(asset_subtree_ids=7681193400980395, limit=1000).to_pandas()
evtest["asset_ids"] = evtest["asset_ids"].apply(lambda x: x[0])
evtest.groupby(["asset_ids"]).size()

In [ ]:
c.assets.retrieve_subtree(id=833579730482990)[0].parent_id

In [ ]:
# alarm_ts = client.events.retrieve_multiple(ids=[9007199140053763, 9007197370256656, 9007194619950911]).to_pandas()
# alarm_metadata = pd.DataFrame(alarm_ts["metadata"].to_list())
# alarm_metadata["external_id"] = alarm_ts["external_id"]
# alarm_metadata